In [16]:
import numpy as np

import jax
from jax import numpy as jnp

import continuousvmc as vmc
from continuousvmc import RotorCNN, VariationalHMC, StochasticReconfiguration, QuantumRotors
from continuousvmc.utils import tree_shape, tree_size

In [2]:
jax.default_backend()

'gpu'

## Wavefunction ansatz definition

In [4]:
logpsi = RotorCNN(dims=(4,4), kernels=[3,3], K=4, param_dtype=jnp.complex64)

In [9]:
key = jax.random.PRNGKey(42)
params = logpsi.initialize(key)

In [14]:
tree_size(params)

656

In [11]:
tree_shape(params)

FrozenDict({
    params: {
        Conv_0: {
            bias: (8,),
            kernel: (3, 3, 8, 8),
        },
        Conv_1: {
            kernel: (3, 3, 8, 1),
        },
    },
})

## The hamiltonian

We look at the Quantum Rotor Model in the continuous basis of rotor angles $\theta$

$$
H \; \mapsto \; \frac{g J}{2} \sum _k \frac{\partial ^2}{\partial \theta _k ^2} - J \sum _{\langle k, l \rangle} \cos(\theta _k - \theta _l)
$$

on the rectangular 4x4 lattice our ansatz `logpsi` is defined on. The associated local energy function is:

$$
E_L [\psi _\alpha] (\theta) \equiv \frac{H \psi _\alpha}{\psi _\alpha} = \frac{g J}{2} \sum _k \frac{1}{\psi _\alpha} \frac{\partial ^2 \psi _\alpha }{\partial \theta _k ^2} - J \sum _{\langle k, l \rangle} \cos(\theta _k - \theta _l)
$$

We can evaluate the local energy in a numerically stable way directly from $\ln \psi _\alpha$ by employing:

$$
\frac{1}{\psi _\alpha} \frac{\partial ^2 \psi _\alpha }{\partial \theta _k ^2} = \frac{\partial ^2 \ln \psi _\alpha }{\partial \theta _k ^2} + \left( \frac{\partial \ln \psi _\alpha }{\partial \theta _k} \right) ^2
$$

and automatic differentiation (AD) from JAX. This is implemented in `QuantumRotors` class, or any local energy class inheriting from `vmc.hamiltonian.LocalEnergy`.

In [6]:
eloc = QuantumRotors(logpsi, g=6.0, pbc=False, chunk_size=4000)

We can evaluate $E_L [\psi _\alpha] (\theta) $ the way you would probably expect:

In [17]:
thetas = np.random.randn(*logpsi.dims)

In [18]:
eloc(params, thetas)

DeviceArray(-4.663691+9.582911j, dtype=complex64)

as well as local energy gradients with respect to ansatz parameters $\alpha$, given samples from $| \psi _\alpha | ^2$ :

In [22]:
dummy_samples = np.random.randn(100, *logpsi.dims)

In [24]:
value, grad = eloc.value_and_grad(params, dummy_samples)

In [25]:
value # The real part

DeviceArray(-8.64675, dtype=float32)

In [26]:
tree_shape(grad)

FrozenDict({
    params: {
        Conv_0: {
            bias: (8,),
            kernel: (3, 3, 8, 8),
        },
        Conv_1: {
            kernel: (3, 3, 8, 1),
        },
    },
})

## Sampling the wavefunction

We use the Hamiltonian Monte Carlo (HMC) algorithm for generating samples from $|\psi _\alpha | ^2$. We simply instantiate:

In [31]:
sampler = VariationalHMC(logpsi, n_samples=100, n_chains=100, warmup=600, n_leaps=50, target_acc_rate=0.8)

and sample using:

In [32]:
key, = jax.random.split(key, 1)
samples, observables, info = sampler(params, key)

In [33]:
samples.shape

(10000, 4, 4)

The resulting samples are returned as an array of shape `(n_chains * n_samples, *dims)`, collecting all samples from all independent chains along the leading axis. Furthermore, the sampler itself is highly customizable to adapt to difficult probability landscapes. Options include: